<a href="https://colab.research.google.com/github/onirosd/python_motor_calidad/blob/main/Ejemplo_de_spark_2_3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !sudo apt update
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
# !wget  https://archive.apache.org/dist/spark/spark-2.3.0/spark-2.3.0-bin-hadoop2.6.tgz   # Using Older Version 

In [3]:
# !tar xf spark-2.3.0-bin-hadoop2.6.tgz
#!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.0-bin-hadoop2.6"

In [5]:
!pip install -q findspark
import findspark
findspark.init()

In [6]:
import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.utils import AnalysisException
from pyspark.sql.functions import col,lit, concat,desc,from_unixtime, when, to_date, unix_timestamp, current_date
from pyspark.sql.window import *
import sys
from datetime import datetime
import funciones as f

#sc = spark.sparkContext
#sc

In [7]:
PRM_RUTA = ''
PRM_NOMBRE = 'DOS'
PRM_SECUENCIA = '2'

CONS_FORMATO_FECHA_RUTINA = "%Y-%m-%d"
CONS_FORMATO_FECHA_MES_ANIO = "%Y%m"
CONS_FORMATO_FECHA_MES_ANIO_DIA = "%Y%m%d"	

In [8]:
rutaConfiguracionRecursos = PRM_RUTA+"JSON_CFG_"+PRM_NOMBRE+"_"+PRM_SECUENCIA+".json"
#spark = eval(f.cargaConfiguracionDeRecursosSpark(rutaConfiguracionRecursos))
#spark.sparkContext.setLogLevel("ERROR")
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab_Diego")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [9]:
rutaConfiguracionProceso = PRM_RUTA+"JSON_PRM_"+PRM_NOMBRE+"_"+PRM_SECUENCIA+".json"
MAP_PROCESS_CONFIGURATION = f.cargaConfiguracionDeParametrosProceso(rutaConfiguracionProceso)

PRM_FECHA_RUTINA = MAP_PROCESS_CONFIGURATION['PRM_SPARK_FECHA_RUTINA'] + ' 00:00:00'

PRM_ESQUEMA_TABLA_INPUT =  MAP_PROCESS_CONFIGURATION['PRM_SPARK_ESQUEMA_TABLA_INPUT']  
#PRM_TABLA_COLUMNAS = MAP_PROCESS_CONFIGURATION['PRM_SPARK_NOMBRE_COLUMNAS']



PRM_SPARK_TABLA_INPUT= MAP_PROCESS_CONFIGURATION['PRM_SPARK_TABLA_INPUT']
#PRM_SPARK_RUTA_FUNCIONES_MOTOR_CALIDAD = MAP_PROCESS_CONFIGURATION['PRM_SPARK_RUTA_FUNCIONES_MOTOR_CALIDAD']
PRM_FECHA_RUTINA_MOTOR = MAP_PROCESS_CONFIGURATION['PRM_SPARK_FECHA_RUTINA']
PRM_SPARK_MOTOR_CALIDAD = MAP_PROCESS_CONFIGURATION['PRM_SPARK_MOTOR_CALIDAD']
PRM_SPARK_RUTACALIDAD = MAP_PROCESS_CONFIGURATION['PRM_SPARK_RUTACALIDAD']
PRM_SPARK_TIPFRECUENCIAREGISTRO = MAP_PROCESS_CONFIGURATION['PRM_SPARK_TIPFRECUENCIAREGISTRO']

In [10]:
from motorcalidad2 import *

In [11]:
def BorrarTemporales(rutaArchivo, nombreCarpeta):
    (ret, out, err) = run_cmd(['hdfs', 'dfs', '-rm', '-r', '-f', "-skipTrash", rutaArchivo + nombreCarpeta])


def F_determinaUniversoCalidad():

    #Gerar codDia
    codDia = datetime.strptime(PRM_FECHA_RUTINA_MOTOR, CONS_FORMATO_FECHA_RUTINA).strftime(CONS_FORMATO_FECHA_MES_ANIO_DIA)
    print('1.1. >>> imprimos dia '+codDia)

	#print(">>> "+str(codDia))
    # Llenar estadisticos para el motor de calidad
    print(PRM_SPARK_MOTOR_CALIDAD+"//"+PRM_SPARK_RUTACALIDAD+"//"+PRM_ESQUEMA_TABLA_INPUT+"//"+PRM_SPARK_TABLA_INPUT+"//"+PRM_FECHA_RUTINA_MOTOR+"//"+PRM_SPARK_TIPFRECUENCIAREGISTRO)
    calculaestadisticos3(spark, PRM_SPARK_MOTOR_CALIDAD, PRM_SPARK_RUTACALIDAD, PRM_ESQUEMA_TABLA_INPUT, PRM_SPARK_TABLA_INPUT, codDia, codDia, PRM_FECHA_RUTINA_MOTOR, PRM_SPARK_TIPFRECUENCIAREGISTRO)
    # intervaloConfianza2(spark, PRM_SPARK_MOTOR_CALIDAD, PRM_ESQUEMA_TABLA_INPUT, PRM_SPARK_TABLA_INPUT, PRM_FECHA_RUTINA_MOTOR, PRM_SPARK_TIPFRECUENCIAREGISTRO)
def main():

    print('1. >>> entramos al main')
    F_determinaUniversoCalidad()
    

In [12]:
main()

1. >>> entramos al main
1.1. >>> imprimos dia 20190728
BCP_DDV_GOBIERNO_MOTORESCALIDAD///desa/bcp/ddv/gobierno/motorescalidad//BCP_UDV_INT//H_TRANSACCIONCONTABLE//2019-07-28//D
+-----------------------+-----------------------+--------------------+--------+--------------------+------+---------------+-----------------------+---------+-----------------+--------------------+-------------+----------------+------------------------------+--------------------------+--------------------------------+-------------------------------+--------------------------+--------------------+-----------------+--------------------+-----------------------+---------------------------------+----------------------------+-----------------+----------------+---------------+--------------------+--------------+--------------+----------------+-----------------+------------------+-----------+---------------+--------------------+--------+--------+--------------------+
|CODCLAVESECUENCIALCAMPO|CODCLAVESECUENCIALREGLA|     

In [13]:
#spark.stop()

In [14]:
#df = spark.read.format("csv").option("delimiter", ";").option("header", "true").load("revisar_excel.csv")

In [15]:
# df2 = df.select(col("NBRCAMPO").alias("nbrcampo"),col("DESREGLA").alias("desregla"),col("CODREGLA").alias("codregla"),col("FLGCALCULOSUMPROM"), col("CODCAMPO"))

# nameFinalTable = 'esquema' + '.' + 'prueba'
# campoanalisis   = ''
# campofrecuencia = ''
# listcadena1 = []
# while df2.count() > 0:
#   column = df2.first()
#   if column['FLGCALCULOSUMPROM'] == '1':
#     cadena = 'SELECT count(' + column["nbrcampo"] + ') AS CTDREGVALORNONULO,' + 'sum(' + column["nbrcampo"] + ') AS NUMVALORSUM,' + 'min(' + column["nbrcampo"] + ') AS NUMVALORMIN,' \
#                      + 'max(' + column["nbrcampo"] + ') AS NUMVALORMAX,' + 'avg(' + column["nbrcampo"] + ') AS NUMVALORPROM,' + 'sum(case when ' + column["nbrcampo"] + ' is null then 1 else 0 end) AS CTDREGVALORNULO, ' + "'" + str(
#                 nameFinalTable) + "'" + ' AS NBRTABLA,' \
#                      + "'" + str(campoanalisis) + "'" + ' AS NUMPERIODO,' + 'count(1) AS CTDTOTALREG, ' + "'" + str(
#                 column["nbrcampo"]) + "'" + ' AS NBRVARIABLE, '  + "'" +str(column["codregla"])+ "'" + ' AS CODCLAVEREGLA, ' + str(column["CODCAMPO"]) + ' AS CODCLAVECAMPO FROM ' + str(nameFinalTable) \
#                      + ' WHERE ' + campofrecuencia + '=' + "'" + str(campoanalisis) + "'" + ' ' + "" + str(column["desregla"]).replace("''","'") + ""
#     listcadena1.append(cadena + ' UNION ALL ')

  
#   else:
#     cad = 'SELECT count(' + column["nbrcampo"] + ') AS CTDREGVALORNONULO, 0 AS NUMVALORSUM,0 AS NUMVALORMIN,0 AS NUMVALORMAX,0 AS NUMVALORPROM,' + 'sum(case when ' + column["nbrcampo"] + ' is null then 1 else 0 end) AS CTDREGVALORNULO, ' + "'" + str(
#                 nameFinalTable) + "'" + ' AS NBRTABLA,' \
#                   + "'" + str(campoanalisis) + "'" + ' AS NUMPERIODO,' + 'count(1) AS CTDTOTALREG, ' + "'" + str(
#                 column["nbrcampo"]) + "'" + ' AS NBRVARIABLE, '+ "'" +str(column["codregla"])+ "'" + ' AS CODCLAVEREGLA, ' + str(column["CODCAMPO"]) + ' AS CODCLAVECAMPO FROM ' + str(nameFinalTable) \
#                   + ' WHERE ' + campofrecuencia + '=' + "'" + str(campoanalisis) + "'" + ' ' + "" + str(column["desregla"]).replace("''","'") + ""
#     listcadena1.append(cad + ' UNION ALL ')


#   df2 = df2.filter(col('nbrcampo') != column['nbrcampo'])
 

